In [1]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import pandas as pd 
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\emreb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
stemmer = SnowballStemmer("english")

In [3]:

data = pd.read_csv('C:/Users/emreb/Documents/projects/newonelast/redditdata//abcnews-date-text.csv', on_bad_lines="skip")
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [4]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [5]:
len(data)

1244184

In [6]:
doc_sample = documents[documents['index'] == 40310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['actor', 'launches', 'coffs', 'green', 'plan']


 tokenized and lemmatized document: 
['actor', 'launch', 'coff', 'green', 'plan']


In [7]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [8]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [9]:
dictionary.filter_extremes(no_below=5, no_above=0.1, keep_n= 100000)

In [10]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[5200]

[(166, 1), (244, 1), (296, 1), (597, 1), (856, 1), (3738, 1), (3739, 1)]

In [11]:
bow_doc_5200 = bow_corpus[5200]
for i in range(len(bow_doc_5200)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_5200[i][0], 
                                               dictionary[bow_doc_5200[i][0]], 
bow_doc_5200[i][1]))

Word 166 ("govt") appears 1 time.
Word 244 ("group") appears 1 time.
Word 296 ("vote") appears 1 time.
Word 597 ("local") appears 1 time.
Word 856 ("want") appears 1 time.
Word 3738 ("compulsori") appears 1 time.
Word 3739 ("ratepay") appears 1 time.


In [12]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5844216176085719),
 (1, 0.38716866963787633),
 (2, 0.5013820927104505),
 (3, 0.5071171375845095)]


In [13]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary,  workers=2,passes=2)

In [14]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.019*"australian" + 0.018*"perth" + 0.017*"miss" + 0.017*"arrest" + 0.013*"tasmanian" + 0.013*"quarantin" + 0.013*"interview" + 0.012*"bank" + 0.011*"hous" + 0.011*"search"
Topic: 1 
Words: 0.024*"news" + 0.019*"market" + 0.018*"canberra" + 0.014*"chang" + 0.013*"australian" + 0.013*"royal" + 0.013*"care" + 0.011*"age" + 0.011*"fall" + 0.011*"climat"
Topic: 2 
Words: 0.044*"queensland" + 0.024*"coronavirus" + 0.021*"coast" + 0.020*"state" + 0.018*"tasmania" + 0.017*"home" + 0.015*"scott" + 0.013*"gold" + 0.012*"andrew" + 0.012*"power"
Topic: 3 
Words: 0.041*"victoria" + 0.036*"elect" + 0.036*"sydney" + 0.023*"crash" + 0.021*"die" + 0.018*"lockdown" + 0.016*"morrison" + 0.012*"guilti" + 0.011*"biden" + 0.011*"road"
Topic: 4 
Words: 0.059*"australia" + 0.054*"covid" + 0.033*"trump" + 0.019*"vaccin" + 0.018*"donald" + 0.018*"test" + 0.017*"world" + 0.016*"record" + 0.016*"coronavirus" + 0.014*"south"
Topic: 5 
Words: 0.021*"health" + 0.020*"nation" + 0.018*"school" + 0.0

Running LDA using TF-IDF

In [15]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.010*"lockdown" + 0.010*"australia" + 0.009*"final" + 0.009*"world" + 0.008*"scott" + 0.006*"open" + 0.006*"beat" + 0.006*"turnbul" + 0.006*"australian" + 0.006*"juli"
Topic: 1 Word: 0.020*"trump" + 0.011*"govern" + 0.010*"countri" + 0.007*"coronavirus" + 0.007*"chang" + 0.007*"hour" + 0.007*"health" + 0.006*"fund" + 0.006*"climat" + 0.006*"say"
Topic: 2 Word: 0.010*"guilti" + 0.009*"jail" + 0.009*"sentenc" + 0.008*"assault" + 0.008*"sexual" + 0.008*"pandem" + 0.007*"david" + 0.007*"quarantin" + 0.007*"mother" + 0.006*"plead"
Topic: 3 Word: 0.013*"restrict" + 0.011*"royal" + 0.010*"coronavirus" + 0.010*"andrew" + 0.009*"commiss" + 0.009*"friday" + 0.008*"age" + 0.008*"thursday" + 0.008*"care" + 0.008*"michael"
Topic: 4 Word: 0.015*"crash" + 0.011*"miss" + 0.010*"polic" + 0.010*"woman" + 0.009*"dead" + 0.009*"search" + 0.008*"death" + 0.008*"kill" + 0.008*"die" + 0.007*"mental"
Topic: 5 Word: 0.010*"live" + 0.009*"coronavirus" + 0.008*"price" + 0.008*"monday" + 0.008*"co

Performance evaluation by classifying sample

In [18]:
for index, score in sorted(lda_model[bow_corpus[5200]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.8874775767326355	 
Topic: 0.031*"govern" + 0.022*"live" + 0.018*"call" + 0.016*"restrict" + 0.014*"island" + 0.013*"water" + 0.013*"return" + 0.012*"alleg" + 0.012*"say" + 0.011*"plan"

Score: 0.012506099417805672	 
Topic: 0.021*"health" + 0.020*"nation" + 0.018*"school" + 0.015*"minist" + 0.015*"busi" + 0.014*"indigen" + 0.013*"communiti" + 0.012*"fund" + 0.011*"hospit" + 0.011*"work"

Score: 0.01250217854976654	 
Topic: 0.028*"face" + 0.025*"peopl" + 0.019*"protest" + 0.016*"street" + 0.013*"citi" + 0.011*"mental" + 0.011*"michael" + 0.011*"turn" + 0.010*"trial" + 0.010*"sport"

Score: 0.012502127327024937	 
Topic: 0.040*"polic" + 0.032*"case" + 0.025*"death" + 0.021*"charg" + 0.021*"year" + 0.020*"court" + 0.019*"murder" + 0.017*"bushfir" + 0.016*"help" + 0.013*"famili"

Score: 0.01250199694186449	 
Topic: 0.019*"australian" + 0.018*"perth" + 0.017*"miss" + 0.017*"arrest" + 0.013*"tasmanian" + 0.013*"quarantin" + 0.013*"interview" + 0.012*"bank" + 0.011*"hous" + 0.011*"sea

Performance evaluation by classifying sample document using LDA TF-IDF 

In [19]:
for index, score in sorted(lda_model_tfidf[bow_corpus[5200]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5084957480430603	 
Topic: 0.020*"trump" + 0.011*"govern" + 0.010*"countri" + 0.007*"coronavirus" + 0.007*"chang" + 0.007*"hour" + 0.007*"health" + 0.006*"fund" + 0.006*"climat" + 0.006*"say"

Score: 0.2068500518798828	 
Topic: 0.026*"news" + 0.015*"rural" + 0.010*"stori" + 0.009*"nation" + 0.009*"sport" + 0.008*"busi" + 0.007*"social" + 0.007*"speak" + 0.007*"insid" + 0.006*"august"

Score: 0.19710545241832733	 
Topic: 0.010*"live" + 0.009*"coronavirus" + 0.008*"price" + 0.008*"monday" + 0.008*"covid" + 0.007*"market" + 0.007*"australian" + 0.007*"record" + 0.006*"cattl" + 0.006*"rain"

Score: 0.012508881278336048	 
Topic: 0.010*"coronavirus" + 0.009*"covid" + 0.009*"elect" + 0.008*"australia" + 0.008*"south" + 0.007*"updat" + 0.006*"wednesday" + 0.006*"violenc" + 0.006*"north" + 0.006*"mark"

Score: 0.012507136911153793	 
Topic: 0.022*"donald" + 0.016*"drum" + 0.010*"weather" + 0.010*"tuesday" + 0.009*"wall" + 0.008*"alan" + 0.008*"christma" + 0.007*"street" + 0.007*"celebr"

### Testing Section

In [20]:
unseen_document= '''I met a traveller from an antique land,
                    Who said—“Two vast and trunkless legs of stone
                    Stand in the desert. . . . Near them, on the sand,
                    Half sunk a shattered visage lies, whose frown,
                    And wrinkled lip, and sneer of cold command,
                    Tell that its sculptor well those passions read
                    Which yet survive, stamped on these lifeless things,
                    The hand that mocked them, and the heart that fed;
                    And on the pedestal, these words appear:
                    My name is Ozymandias, King of Kings;'''

print(unseen_document)
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

I met a traveller from an antique land,
                    Who said—“Two vast and trunkless legs of stone
                    Stand in the desert. . . . Near them, on the sand,
                    Half sunk a shattered visage lies, whose frown,
                    And wrinkled lip, and sneer of cold command,
                    Tell that its sculptor well those passions read
                    Which yet survive, stamped on these lifeless things,
                    The hand that mocked them, and the heart that fed;
                    And on the pedestal, these words appear:
                    My name is Ozymandias, King of Kings;
Score: 0.1953313648700714	 Topic: 0.023*"attack" + 0.023*"china" + 0.022*"warn" + 0.019*"north" + 0.018*"kill"
Score: 0.14337611198425293	 Topic: 0.024*"news" + 0.019*"market" + 0.018*"canberra" + 0.014*"chang" + 0.013*"australian"
Score: 0.14110662043094635	 Topic: 0.059*"australia" + 0.054*"covid" + 0.033*"trump" + 0.019*"vaccin" + 0.018*"donald"
Score: 

In [21]:
from gensim.models import CoherenceModel

In [22]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(bow_corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
# coherence_model_lda = CoherenceModel(model= lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.126624749097992


In [23]:
# import zipfile
import pyLDAvis
import pyLDAvis.gensim 

c:\Users\emreb\Documents\projects\translatortelegram\transtele\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [26]:
pyLDAvis.enable_notebook()
visal = pyLDAvis.gensim.prepare(lda_model,  bow_corpus,dictionary)
visal
# Each bubble on the left-hand side plot represents a topic. The larger the bubble, the more prevalent is that topic.
# A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.

c:\Users\emreb\Documents\projects\translatortelegram\transtele\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.138015 -0.206994       1        1  11.819380
8     -0.215343 -0.136268       2        1  11.738042
5     -0.207184 -0.116560       3        1  11.661939
4     -0.027844  0.183311       4        1  11.126157
2     -0.118774 -0.124284       5        1   9.821592
1     -0.101831  0.164541       6        1   9.609325
9      0.079375  0.205018       7        1   9.187837
3      0.173685 -0.107439       8        1   8.835280
0     -0.000433  0.193455       9        1   8.539340
7      0.280333 -0.054780      10        1   7.661107, topic_info=             Term          Freq         Total Category  logprob  loglift
38      australia  37816.000000  37816.000000  Default  30.0000  30.0000
25345       covid  35055.000000  35055.000000  Default  29.0000  29.0000
2051   queensland  24587.000000  24587.000000  Default  28.0000  28.0000
240         polic  32355.000000  32355.000000  Default  27.0000  27.0000
1505     victoria  20771.000000  20771.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
946         ralli   2809.384689   2817.006077  Topic10  -5.0554   2.5663
348       protest   8556.889194  12373.656372  Topic10  -3.9416   2.2002
488         trial   4493.835240   9013.623759  Topic10  -4.5857   1.8730
2211       sydney   4345.895185  22450.537736  Topic10  -4.6191   0.9269
863      melbourn   3408.451164  16349.806836  Topic10  -4.8621   1.0011

[423 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
326       3  0.999851  aborigin
1939      1  0.802576      abus
1939      3  0.197202      abus
215       1  0.978506     accus
215      10  0.021366     accus
...     ...       ...       ...
1127      4  0.250883      year
361       6  0.999752     young
117       3  0.999301     youth
117      10  0.000303     youth
388       4  0.999756   zealand

[510 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 9, 6, 5, 3, 2, 10, 4, 1, 8])

In [ ]:
#  results shows consistency that's nice

 TFI-IDF Table

In [29]:
visaltf_idf = pyLDAvis.gensim.prepare(lda_model_tfidf,  corpus_tfidf,dictionary)
visaltf_idf

c:\Users\emreb\Documents\projects\translatortelegram\transtele\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.051715  0.176405       1        1  14.305216
0      0.161969 -0.302183       2        1  10.879381
6      0.053510 -0.022101       3        1  10.387424
4     -0.178447 -0.067883       4        1   9.998485
5      0.083371  0.025971       5        1   9.762801
7     -0.241705 -0.047624       6        1   9.565238
2     -0.225249  0.036398       7        1   9.183125
3      0.092743  0.112134       8        1   8.866613
9      0.127701  0.120071       9        1   8.716305
8      0.074391 -0.031188      10        1   8.335413, topic_info=            Term         Freq         Total Category  logprob  loglift
7337       trump  7285.000000   7285.000000  Default  30.0000  30.0000
1281        news  5765.000000   5765.000000  Default  29.0000  29.0000
1568      donald  4685.000000   4685.000000  Default  28.0000  28.0000
2951        drum  3397.000000   3397.000000  Default  27.0000  27.0000
197        crash  4275.000000   4275.000000  Default  26.0000  26.0000
...          ...          ...           ...      ...      ...      ...
2029       pacif   923.362229   1666.216354  Topic10  -5.4506   1.8944
2121        wild   814.310383   1181.059723  Topic10  -5.5763   2.1128
1287       great   843.704864   1443.573494  Topic10  -5.5408   1.9476
38     australia   877.448956  11725.958135  Topic10  -5.5016  -0.1079
16    australian   833.474052   8808.364608  Topic10  -5.5530   0.1268

[567 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
2586      3  0.157964   abbott
2586      7  0.772960   abbott
2586      8  0.012424   abbott
2586      9  0.055909   abbott
1939      1  0.051974     abus
...     ...       ...      ...
1953      2  0.998399     york
388       2  0.716697  zealand
388       3  0.178727  zealand
388       5  0.104258  zealand
8325      8  0.998132     zero

[1444 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 7, 5, 6, 8, 3, 4, 10, 9])

## Salient Topics

In [30]:
sal_lda_model = gensim.models.ldamodel.LdaModel(corpus=bow_corpus,
                                           id2word=dictionary,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
print('\nPerplexity: ', sal_lda_model.log_perplexity(bow_corpus)) 

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(sal_lda_model, bow_corpus, dictionary)
vis

In [ ]:
# Print the Keyword in the 10 topics
# pprint(lda_model.print_topics())
# doc_lda = lda_model[dictionary]

In [ ]:
import zipfile
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip

In [ ]:
with zipfile.ZipFile('/mallet-2.0.8.zip', 'r') as zip_ref:#the directory of zipped file
    zip_ref.extractall('C:/Users/emreb/Documents/projects/newonelast') #directory of unzipped file

In [ ]:
import os

os.environ.update({
        'MALLET_HOME':
        r"/content/drive/MyDrive/dataAI/mallet/mallet-2.0.8/bin/mallet",
    })

In [ ]:
mallet_path = '/content/drive/MyDrive/dataAI/mallet/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)